In [1]:
import pandas as pd
import os

In [2]:
path = os.path.join("Resources", "crime_incident_data2017.csv")

In [3]:
df = pd.read_csv(path)\
    .rename(columns=lambda s: s.lower().replace(" ", "_"))\
    [[
    'neighborhood',
    'occur_date',
    'occur_time',
    'offense_category',
    'offense_count',
    'offense_type',
    'report_date'
]].dropna()

In [4]:
df.columns

Index(['neighborhood', 'occur_date', 'occur_time', 'offense_category',
       'offense_count', 'offense_type', 'report_date'],
      dtype='object')

In [5]:
df.loc[df['offense_count'] >= 2, "offense_category"].unique()

array(['Motor Vehicle Theft'], dtype=object)